In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# подключаем библиотеку для построения линейной регрессии
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso, Ridge

In [54]:
#загружаем набор данных и выводим 5 первых строк
bunch = pd.read_csv("day.csv")
bunch.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [10]:
#проверяем числог строк и столбцов в наборе данных
bunch.shape

(731, 16)

In [11]:
#проверяем заполнение строк и типы признаков
bunch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
instant       731 non-null int64
dteday        731 non-null object
season        731 non-null int64
yr            731 non-null int64
mnth          731 non-null int64
holiday       731 non-null int64
weekday       731 non-null int64
workingday    731 non-null int64
weathersit    731 non-null int64
temp          731 non-null float64
atemp         731 non-null float64
hum           731 non-null float64
windspeed     731 non-null float64
casual        731 non-null int64
registered    731 non-null int64
cnt           731 non-null int64
dtypes: float64(4), int64(11), object(1)
memory usage: 91.5+ KB


In [23]:
#указываем входы и выходы модели
X = bunch.drop(['cnt','dteday', 'instant', 'casual','registered'], axis=1) #выбрасываем столбец cnt из входа; axis если =0 - строка, если = 1 -столбец
#ID выбросили, потому что не влияет на результаты предсказания
y = bunch['cnt']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [25]:
X_train.shape, X_test.shape

((584, 11), (147, 11))

In [26]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test) # считаем коэффициент детерминации

0.7359320083715085

In [27]:
#среднеквадратическая ошибка
mean_squared_error(y_test, lr.predict(X_test))

902586.8092821774

In [28]:
#выводим коэффициенты b и k из уравнения y=kx+b; в лекциях b - w0, k - w1
lr.intercept_, lr.coef_

(1373.777310891206,
 array([  515.25932904,  2057.3430526 ,   -42.05852397,  -630.18111999,
           74.36599375,   148.46987073,  -617.901092  ,  2061.93280057,
         3634.06338811, -1053.9594686 , -2194.82730878]))

In [29]:
#проверяем, нельзя ли улучшить линейную регрессию, добавив полиномиальные признаки
pf = PolynomialFeatures(degree=2)
X_train_polynomial = pf.fit_transform(X_train)
X_test_polynomial = pf.fit_transform(X_test)
lf = LinearRegression()
lf.fit(X_train_polynomial, y_train)
lf.score(X_train_polynomial, y_train) #смотрим R^2 на обучающей выборке

0.9101635768454955

In [30]:
#смотрим R^2 на тестовой выборке
lf.score(X_test_polynomial, y_test)

0.8331023165147384

In [31]:
from sklearn.linear_model import Lasso, Ridge
lf = Ridge(fit_intercept=True, alpha=0.001, normalize=True)
lf.fit(X_train_polynomial, y_train)

Ridge(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001)

In [32]:
lf.score(X_test_polynomial, y_test)

0.8291150973403003

In [51]:
lf = Lasso(fit_intercept=True, alpha=0.007, normalize=True)
lf.fit(X_train_polynomial, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Lasso(alpha=0.007, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [52]:
lf.score(X_test_polynomial, y_test)

0.8307726047313885